# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
import random
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore') 

nltk.download(['words', 'punkt', 'stopwords', 'averaged_perceptron_tagger', 'maxent_ne_chunker', 'wordnet'])
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, ne_chunk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
# Creating two separated dataframes
# X is the dataframe that contains the dependent column to be predicted : 'message'
X = df['message']
# Y is the dataframe that contains the independent columns (all binary)
Y = df.iloc[:,4:]
#Check the dimension of the X and Y dataframes
X.shape, Y.shape

((26028,), (26028, 36))

In [5]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X.head(3)

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    #check for urls in the text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text) 
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # normalize the text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    #tokenize the text 
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    
    #lemmatize the text
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

In [10]:
# Check to see if the tokenize function works properly
n = random.randint(0,len(X))
print('The original version:', X[n])
print('')
print('The tokenized version:', tokenize(X[n]))

The original version: Further to the east, into Liberia, Code d'Ivoire, and southwestern Guinea, seasonable dry conditions continue.

The tokenized version: ['further', 'east', 'liberia', 'code', 'ivoire', 'southwestern', 'guinea', 'seasonable', 'dry', 'condition', 'continue']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=0)))
    ])
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fd3abb2b0d0>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=0, ve

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
# Creation the train and test sets for the independent and dependent variables dataframes
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [13]:
# Display the dimension of all 4 dataframes
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)
print('y_train shape: ', y_train.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (19521,)
X_test shape:  (6507,)
y_train shape:  (19521, 36)
y_test shape:  (6507, 36)


In [14]:
X_train.head(3)

1394               How can I go to the Dominican Republic?
20329    Other areas vulnerable to a poor belg season w...
1747     hello i want to know what flights are leaving ...
Name: message, dtype: object

In [15]:
X_test.head(3)

17757    A truck carrying three men also careened into ...
13468    CHENGDU, July 11 (Xinhua) -- Sixteen bodies ha...
6160     I don't where I need to go to get help. I need...
Name: message, dtype: object

In [16]:
y_train.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
1394,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20329,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1747,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
y_test.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
17757,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13468,1,0,0,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
6160,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [19]:
#test the model 
y_pred = pipeline.predict(X_test)
print(y_pred)

[[1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]


In [21]:
# Source: https://stackoverflow.com/questions/56826216/valueerror-unknown-label-type-for-classification-report
#from sklearn.utils.multiclass import type_of_target
#type_of_target(y_test)
#type_of_target(y_pred)

'multilabel-indicator'

In [20]:
print(classification_report(y_test, y_pred, target_names =Y.columns)) 

                        precision    recall  f1-score   support

               related       0.85      0.92      0.88      4947
               request       0.81      0.43      0.57      1138
                 offer       0.00      0.00      0.00        34
           aid_related       0.76      0.60      0.67      2743
          medical_help       0.52      0.08      0.14       514
      medical_products       0.77      0.07      0.12       341
     search_and_rescue       0.69      0.09      0.16       203
              security       0.00      0.00      0.00       128
              military       0.57      0.07      0.12       237
           child_alone       0.00      0.00      0.00         0
                 water       0.84      0.37      0.51       426
                  food       0.85      0.47      0.61       744
               shelter       0.80      0.32      0.46       601
              clothing       0.70      0.13      0.22       105
                 money       0.78      

In [32]:
accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :",accuracy)

The calculated accuracy is : related                   0.814815
request                   0.883510
offer                     0.994775
aid_related               0.749501
medical_help              0.921469
medical_products          0.950054
search_and_rescue         0.970340
security                  0.980329
military                  0.964192
child_alone               1.000000
water                     0.954203
food                      0.930690
shelter                   0.929768
clothing                  0.985093
money                     0.978024
missing_people            0.986937
refugees                  0.964192
death                     0.958967
other_aid                 0.863532
infrastructure_related    0.931458
transport                 0.952820
buildings                 0.954664
electricity               0.982480
tools                     0.992316
hospitals                 0.990011
shops                     0.995851
aid_centers               0.986783
other_infrastructure      

### 6. Improve your model
Use grid search to find better parameters. 

In [33]:
parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

{'clf__estimator__n_estimators': [50, 100, 200],
 'clf__estimator__min_samples_split': [2, 3, 4]}

In [34]:
cv = GridSearchCV(pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...1,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [50, 100, 200], 'clf__estimator__min_samples_split': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)
Y_pred

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
print(classification_report(y_test, y_pred, target_names =Y.columns))

In [ ]:
accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#Decision tree
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier(()))
    ])
pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

cv = GridSearchCV(pipeline, param_grid=parameters)
cv


cv.fit(X_train, y_train)


y_pred = cv.predict(X_test)
Y_pred

print(classification_report(y_test, y_pred, target_names =Y.columns))

accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

In [ ]:
#KNN
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])
pipeline.get_params()

#Decision tree
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', DecisionTreeClassifier(()))
    ])
pipeline.get_params()

parameters = {
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4]
    }
parameters

cv = GridSearchCV(pipeline, param_grid=parameters)
cv


cv.fit(X_train, y_train)


y_pred = cv.predict(X_test)
Y_pred

print(classification_report(y_test, y_pred, target_names =Y.columns))

accuracy = (y_pred == y_test).mean()
print("The calculated accuracy is :", accuracy)

### 9. Export your model as a pickle file

In [ ]:
#source:Grepper.com
# save the model to disk
filename = 'Disaster_model.pkl'
pickle.dump(model, open(filename, 'wb'))

# load the model from disk
model = pickle.load(open(filename, 'rb'))
result = model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.